# Моделирование влияния внешних факторов на вид $\pi$-A изотермы сжатия двумерного газа

Alexander Sorokin, Ivanovo, Russia; <a href = "mailto:a40in@mail.ru">a40in@mail.ru</a>. <font size=-1>July 26th, 2022</font>

**Simulation of pressure-area isotherm of 2D gas under external influence**

<font size=-1>A Volmer-type pressure-area isotherm in the presence of noise, zero shift and zero drift at varying temperature is modeled to understand how these factors affect the appearance of the isotherm.</font>

Частным случаем изотермы сжатия реального двумерного газа является изотерма, описываемая уравнением Фольмера (изотерма Фольмера). Анализ изотермы, полученной в эксперименте, с использованием этого уравнения дает возможность определять параметры двумерного газа и применяется на практике. В реальном измерении получаемые данные искажаются шумом (собственным шумом датчика поверхностного давления, внешней вибрацией и т.п.); сдвигом нуля датчика, который может меняться в ходе записи изотермы; непостоянством температуры; другими факторами.

Здесь моделируется изотерма Фольмера, искаженная рядом указанных факторов. Генерируется массив данных, соответствующих изотерме Фольмера при 298 К (идеальная изотерма). Затем к этой идеальной изотерме добавляются помехи (шум, постоянный сдвиг нуля, линейно изменяющийся сдвиг нуля), меняется температура (формируется реальная изотерма). Идеальная и реальная изотермы отображаются на одном графике. Модель создана для того, чтобы понять, как различные факторы влияют на вид реальной изотермы Фольмера.

В плане программирования эта работа представляет пример создания в Jupyter Notebook интерактивного графика. Данные, отображаемые на графике, вычисляются по некоторой формуле. Значения параметров в этой формуле можно менять, используя визуальные элементы управления и мышь (вместо ручного ввода с клавиатуры).

IDEAL GAS

$ \pi S =\nu RT $ 

$ \pi S (\pi) $ is constant ($ \pi S$ is independent of $\pi$. At 1 nanomole, 300 K $ \pi S = 2.5 \: \mu J$, in other words, $S = 25\:cm^2$ at $1\:mN/m, 2.5\:cm^2$ at $10\:mN/m)$.

VOLMER GAS

$ \pi(S-S_0)=\nu RT $

$ S_0 = \xi_0N = \xi_0 \nu N_A $

$ \pi S = \nu RT + \pi \xi_0 \nu N_A  $

$ \pi S (\pi) $ should be linear, $ y = ax + b $, with the slope $ a = \xi_0 \nu N_A $ and vertical shift $ b = \nu RT \:(=2.5\:\mu J$ at 1 nmole, $300 \: K$)

A possible alternative is to plot $ \frac{\pi S}{RT} $ vs $ \pi$: $\frac{\pi S}{RT} = \nu + \nu\frac{\xi_0}{kT}\pi$

REAL GAS

$ (\pi - \frac{\alpha}{S^2})(S-S_0)=\nu RT $

In [3]:
# КОД 

import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets

def update_plot():
    S, pd, pdS = iso_and_volmer(tempe, noise, shift, tilto)
    real_iso.set_xdata(S)
    real_iso.set_ydata(pd)
    real_volmer.set_xdata(pd)
    real_volmer.set_ydata(pdS)
    ax_1.relim()
    ax_1.autoscale_view()
    ax_2.relim()
    ax_2.autoscale_view()
    fig.canvas.draw()
    
def update_tempe(change):
    global tempe
    tempe = change.new
    update_plot()

def update_noise(change):
    global noise
    noise = change.new
    update_plot()
    
def update_shift(change):
    global shift
    shift = change.new
    update_plot()

def update_tilto(change):
    global tilto
    tilto = change.new
    update_plot()    
    
def iso_and_volmer(T, noise_amplitude, shift_value, shift_slope):
    A = np.linspace(0.005, 0.1, 200)
    slope = np.linspace(0, shift_slope, 200)
    bias = np.random.uniform(-1*noise_amplitude, noise_amplitude, len(A)) + shift_value + slope
    pi = 1000 * 1e-6*eta*R*T / (A - 1e-18*ksi*1e-6*eta*NA) + bias
    piA = 1000*pi*A
    return A, pi, piA

R = 8.31 # J/(mol K)
Temp = 273  # K
NA = 6.02e23 # mol^-1
eta = 0.005 # micromole
ksi = 0.25 # nm^2
    
tempe = 298.
noise = 0.
shift = 0.
tilto = 0.

tempe_slider = widgets.IntSlider(
               value=tempe, min=273, max=323, step=1,
               description='Temperature ', continuous_update=False)

noise_slider = widgets.FloatSlider(
               value=noise, min=0, max=2, step=0.05,
               description='Noise ', continuous_update=False)

shift_slider = widgets.FloatSlider(
               value=shift, min=-2, max=2, step=0.05,
               description='Shift ', continuous_update=False)

tilto_slider = widgets.FloatSlider(
               value=tilto, min=-2, max=2, step=0.05,
               description='Tilt ', continuous_update=False)

tempe_slider.observe(update_tempe, 'value')
noise_slider.observe(update_noise, 'value')
shift_slider.observe(update_shift, 'value')
tilto_slider.observe(update_tilto, 'value')

%matplotlib widget
fig = plt.figure()
fig.set_facecolor('#e0e0e0')
fig.canvas.header_visible = False
fig.canvas.resizable = False
fig.set_size_inches(9.8, 4.8) #default 6.4, 4.8

ax_1 = fig.add_subplot(1, 2, 1)
ax_1.grid()
ax_1.set_title('Изотерма (мН/м от м$^2$)')
ax_2 = fig.add_subplot(1, 2, 2)
ax_2.grid()
ax_2.set_title('График Фольмера (мкДж от мН/м)')

S, pd, pdS = iso_and_volmer(298, 0, 0, 0)
ideal_iso, = ax_1.plot(S, pd)
ideal_volmer, = ax_2.plot(pd, pdS)

S, pd, pdS = iso_and_volmer(tempe, noise, shift, 0)
real_iso, = ax_1.plot(S, pd, "o", markersize = 2)
real_volmer, = ax_2.plot(pd, pdS, "o", markersize = 2)

widgets.VBox([tempe_slider, noise_slider, shift_slider, tilto_slider])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

**ЛИТЕРАТУРА**

<ins>Физика</ins>

1. E. K. Rideal. An Introduction to Surface Chemistry. 2nd ed. Cambridge, 1930 (Э. К. Райдил. Химия поверхностных явлений. Л., ОНТИ-Химтеорет, 1936).
2. N. K. Adam. The Physics and Chemistry of Surfaces. Clarendon Press, Oxford, 1938 (Н. К. Адам. Физика и химия поверхностей. М. ОГИЗ 1947).
3. A. Adamson. Physical Chemistry of Surfaces. Wiley 19?? (3rd ed.) (А. Адамсон. Физическая химия поверхностей. М. Мир, 1979).

<ins>Программирование</ins>

*Matplotlib*

0. https://matplotlib.org
1. Хорошо поданные основы https://pyprog.pro/mpl/mpl_short_guide.html и https://jenyay.net/Matplotlib/Matplotlib
14. Пособие по Mathplotlib. https://matplotlib.org/stable/tutorials/index.html
17. Краткое руководство по pyplot https://pyprog.pro/mpl/mpl_main_components.html

*Интерактивность в юпитерианских блокнотах*

1. https://habr.com/ru/post/485318/
2. https://medium.com/nuances-of-programming/интерактивное-управление-в-jupyter-notebooks-4fd1fccb5788
3. Jupiter Widgets (ipywidgets) https://ipywidgets.readthedocs.io/en/stable/index.html
4. ipympl (библиотека, обеспечивающая интерактивность) https://matplotlib.org/ipympl/, особенно вот это https://matplotlib.org/ipympl/examples/full-example.html
5. https://kapernikov.com/ipywidgets-with-matplotlib/, https://github.com/Kapernikov/ipywidgets-tutorial/blob/master/ipywidgets-tutorial.ipynb

*Разметка*
1. LaTeX https://grammarware.net/text/syutkin/MathInLaTeX.pdf
2. Markdown https://www.markdownguide.org/basic-syntax/ 